## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_dfs for the Attrition and Department columns
y_attrition = pd.DataFrame(attrition_df['Attrition'])
y_department = pd.DataFrame(attrition_df['Department'])

In [4]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse=False, drop='first')

# Fit and apply the encoder to the department data
y_department_encoded = encoder.fit_transform(y_department[['Department']])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Create a OneHotEncoder for the Attrition column
a_encoder = OneHotEncoder(sparse=False)

# Fit and apply the encoder to the training data
y_attrition_encoded = encoder.fit_transform(y_attrition[['Attrition']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# Create a list of at least 10 column names to use as X data
columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel',
                     'WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked'
                     ]
# Create X_df using your selected columns
X_df = attrition_df[columns]

# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_department_train, y_department_test = train_test_split(
    X_df,
    y_department_encoded,
    test_size=0.2,
    random_state=42
)
X_train, X_test, y_attrition_train, y_attrition_test = train_test_split(
    X_df,
    y_attrition_encoded,
    test_size=0.2,
    random_state=42
)


In [8]:
# Check shape of y trainsc
y_attrition_train.shape

(1176, 1)

In [9]:
y_department_train.shape

(1176, 2)

In [10]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})


In [11]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data
num_features = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared_hidden_layer1 = layers.Dense(128, activation='relu')(input_layer)
shared_hidden_layer2 = layers.Dense(64, activation='relu')(shared_hidden_layer1)

In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu', name='department_hidden')(shared_hidden_layer2)

# Create the output layer
department_output = layers.Dense(2, activation='softmax', name='department_output')(department_hidden_layer)


In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu', name='attrition_hidden')(shared_hidden_layer2)


# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)



In [15]:
import tensorflow as tf

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
    metrics={'department_output': ['accuracy'], 'attrition_output': ['accuracy']}
)
# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 128)                  1408      ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 department_hidden (Dense)   (None, 32)                   2080      ['dense_1[0][0]']             
                                                                                              

In [16]:
# Train the model
model.fit(
    X_train,
    {'department_output': y_department_train, 'attrition_output': y_attrition_train},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
30/30 [==============================] - 7s 73ms/step - loss: 1.1474 - department_output_loss: 0.6782 - attrition_output_loss: 0.4692 - department_output_accuracy: 0.6351 - attrition_output_accuracy: 0.8362 - val_loss: 1.1677 - val_department_output_loss: 0.6818 - val_attrition_output_loss: 0.4860 - val_department_output_accuracy: 0.6695 - val_attrition_output_accuracy: 0.7966
Epoch 2/10
30/30 [==============================] - 0s 15ms/step - loss: 1.0600 - department_output_loss: 0.6402 - attrition_output_loss: 0.4198 - department_output_accuracy: 0.6511 - attrition_output_accuracy: 0.8404 - val_loss: 1.1769 - val_department_output_loss: 0.6973 - val_attrition_output_loss: 0.4795 - val_department_output_accuracy: 0.6695 - val_attrition_output_accuracy: 0.7966
Epoch 3/10
30/30 [==============================] - 0s 13ms/step - loss: 1.0388 - department_output_loss: 0.6069 - attrition_output_loss: 0.4319 - department_output_accuracy: 0.6638 - attrition_output_accuracy: 0.8404 

In [19]:
# Evaluate the model with the testing data and print the accuracy for both department and attrition
results = model.evaluate(
    X_test,
    {'department_output': y_department_test, 'attrition_output': y_attrition_test},
    batch_size=32
)
print("Model evaluation results:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value}")

10/10 [==============================] - 0s 3ms/step - loss: 1.0821 - department_output_loss: 0.7209 - attrition_output_loss: 0.3612 - department_output_accuracy: 0.7109 - attrition_output_accuracy: 0.8605
Model evaluation results:
loss: 1.0821170806884766
department_output_loss: 0.720909833908081
attrition_output_loss: 0.36120736598968506
department_output_accuracy: 0.7108843326568604
attrition_output_accuracy: 0.8605442047119141


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
2. What activation functions did you choose for your output layers, and why?
3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is an okay metric for this dataset. However, a more robust evaluation could be performed with the F1 score taking into account both false positivies and negatives. Balanced accuracy could be considered as well given the multiclass nature of the department output.
2.  I used a sigmoid function for the attrition output layer as it is a binary task and the sigmoid function gives a probability score between 0 and 1. I used a softmax function for the department output layer as it is a multiclass task and the softmax function gives a probability distribution across classes.
3. hyperparameter tuning, further training epochs, advanced architectures, data augmentation, feature engineering.